# Champions general statistics

Get the saved games data

In [1]:
import json, os
gamesRawData = []
for i in os.listdir("./games"):
    if i.endswith(".json"):
        with open("./games/"+i,"r") as f:
            gamesRawData.append(json.loads(f.read()))

Initialize the champion stats data structure.

In [2]:
import requests

response = requests.get('http://ddragon.leagueoflegends.com/cdn/8.19.1/data/en_US/champion.json')
champonRawData = json.loads(response.text)

championsStats = {}
for key,champion in champonRawData['data'].items():
    championsStats[int(champion['key'])] = {
                    "champion":champion['name'],
                    "games":0,
                    "gamesPlayed":0,
                    "wins":0,
                    "teamKills":0,
                    "kills":0,
                    "deaths":0,
                    "assists":0,
                    "bans":0
                }


Parse all the games to get statistics by champion

In [3]:
for game in gamesRawData:
    #Count the champion bans
    for t in game["teams"]:
        for b in t["bans"]:
            championsStats[b["championId"]]["bans"] += 1
            championsStats[b["championId"]]["games"]+=1
    
    #Count the kills of the team to later get the kill participation
    teamKills = {100:0,200:0}
    for p in game["participants"]:
        teamKills[p["teamId"]] += p["stats"]["kills"]
    
    #Count the wins and basic stats like kills, deaths and assits
    for p in game["participants"]:
        championsStats[p["championId"]]["games"]+=1
        championsStats[p["championId"]]["gamesPlayed"]+=1
        if p["stats"]["win"]:
            championsStats[p["championId"]]["wins"] += 1
        
        championsStats[p["championId"]]["teamKills"] += teamKills[p["teamId"]]
        championsStats[p["championId"]]["kills"] += p["stats"]["kills"]
        championsStats[p["championId"]]["deaths"] += p["stats"]["deaths"]
        championsStats[p["championId"]]["assists"] += p["stats"]["assists"]

We make a difference between games and gamesPlayed where games counts every game where the champion has been either played or banned.

Use pandas Dataframe to manipulate the data

In [4]:
import pandas as pd, numpy as np
df = pd.DataFrame(championsStats).T

#Get the winrate for each champion
#wr is the raw winrate
df["wr"] = 0
df.loc[df["gamesPlayed"]>0,"wr"] = df.loc[df["gamesPlayed"]>0]["wins"]/df.loc[df["gamesPlayed"]>0]["gamesPlayed"]
#winrate is the beautified winrate
df["winrate"] = (df["wr"] * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

#Get the KDA
df["KDA"] = 0
df.loc[df["deaths"]>0,"KDA"] = ((df.loc[df["deaths"]>0]["kills"] + df.loc[df["deaths"]>0]["assists"]) / df.loc[df["deaths"]>0]["deaths"])
#Handling the case when KDA is infinite (deaths=0)
df.loc[df["deaths"]==0,"KDA"]=(df.loc[df["deaths"]==0]["kills"] + df.loc[df["deaths"]==0]["assists"])
#Beautify KDA
df["KDA"] = df["KDA"].map('{:,.2f}'.format)

#Get kill participation and beautify it
df["KP"] = 0
df.loc[df["teamKills"]>0,"KP"] = (((df.loc[df["teamKills"]>0]["kills"] + df.loc[df["teamKills"]>0]["assists"]) / df.loc[df["teamKills"]>0]["teamKills"]) * 100).map('{:,.2f}'.format).apply(lambda s: str(s)+"%")

Resulting Dataframe

In [5]:
df

,assists,bans,champion,deaths,games,gamesPlayed,kills,teamKills,wins,wr,winrate,KDA,KP
1,0,0,Annie,0,0,0,0,0,0,0,0.00%,0.00,0
2,46,18,Olaf,24,27,9,19,88,3,0.333333,33.33%,2.71,73.86%
3,30,10,Galio,10,16,6,17,70,3,0.5,50.00%,4.70,67.14%
4,0,0,Twisted Fate,0,0,0,0,0,0,0,0.00%,0.00,0
5,22,4,Xin Zhao,19,10,6,9,54,3,0.5,50.00%,1.63,57.41%
6,85,23,Urgot,41,42,19,66,269,13,0.684211,68.42%,3.68,56.13%
7,48,15,LeBlanc,20,25,10,38,137,6,0.6,60.00%,4.30,62.77%
8,0,0,Vladimir,0,0,0,0,0,0,0,0.00%,0.00,0
9,0,0,Fiddlesticks,0,0,0,0,0,0,0,0.00%,0.00,0
10,0,0,Kayle,0,0,0,0,0,0,0,0.00%,0.00,0


Some manipulation examples

In [6]:
(df[["champion","kills","deaths","assists","KDA","teamKills","KP","wins","gamesPlayed","games","bans","winrate","wr"]] #Selecting the columns to show and their order
    .sort_values(["wr","gamesPlayed"], ascending=False) #Sorting champions by winrate and games played
    .drop("wr", axis=1)) #Drop the "wr" column which is only here for sorting purpose, the "winrate" column giving the same information but beautified

,champion,kills,deaths,assists,KDA,teamKills,KP,wins,gamesPlayed,games,bans,winrate
202,Jhin,16,2,21,18.50,50,74.00%,3,3,4,1,100.00%
43,Karma,3,2,16,9.50,24,79.17%,2,2,3,1,100.00%
111,Nautilus,2,1,21,23.00,37,62.16%,2,2,2,0,100.00%
268,Azir,8,0,9,17.00,25,68.00%,2,2,4,2,100.00%
26,Zilean,2,0,3,5.00,5,100.00%,1,1,5,4,100.00%
27,Singed,2,2,7,4.50,18,50.00%,1,1,3,2,100.00%
34,Anivia,3,0,12,15.00,18,83.33%,1,1,5,4,100.00%
41,Gangplank,2,3,10,4.00,14,85.71%,1,1,1,0,100.00%
122,Darius,1,1,5,6.00,11,54.55%,1,1,3,2,100.00%
154,Zac,3,1,11,14.00,18,77.78%,1,1,1,0,100.00%


In [7]:
#Getting percentage of champions either played or banned in Worlds 2018
df[df["games"] > 0].shape[0] / df.shape[0]

0.574468085106383

In [ ]:
S